# Calculating correlation and skill score on C3S seasonal forecasts
This notebook shows an example workflow on how to compute correlation and probabilistic skill scores with open source Python tools. 
This notebook has three sections:

1.   How to prepare the environment
2.   How to download the data from the CDS
3.   The calculation of the scores

This notebook has the aim to show the potentiality of a xarray-based workflow starting from the CDS data. The metrics shown are calculated in a naive way, without applying bias correction or any type of calibration.

**Important note**: to download the files from the Copernicus Data Store (CDS) you need to use your API key ([see here](https://cds.climate.copernicus.eu/api-how-to))

Author: [Matteo De Felice](http://matteodefelice.name) - [github](https://github.com/matteodefelice/c3s-xskillscore/)

**UPDATE (March 2021)** Cartopy it doesn't install any more on Google Colab and my calculation of the Brier Skill Score is not compatible any more with the `xskillscore` functions (see https://github.com/xarray-contrib/xskillscore/issues/49)



## 1. Preparing the environment
### 1.1 ecCodes and cartopy with apt
The ECMWF ecCodes binary library is needed by `cfgrib` and the `cartopy` module is useful for maps (we prefer installing with ``apt-get`` due to potential issues with the version provided by Anaconda).

In [1]:
!apt-get -qq install -y libeccodes0 python-cartopy python3-cartopy

zsh:1: command not found: apt-get


### 1.2 Installing Anaconda
The module `esmpy` is not available via pip and will be installed from the `conda-forge` channel. All the other modules (except `cdsapi`) will be installed as well via anaconda. We follow the steps suggested in https://rjai.me/posts/google-colab-conda/

In [2]:
!wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh && bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local


--2022-08-31 19:39:30--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-08-31 19:39:30--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  73.06M  14.5MB/s    in 4.5s    

2022-08-31 19:39:34 (16.4 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [76607678/76607678]

PREFIX=/usr/lo

In [3]:
!conda install -y -q --prefix /usr/local -c conda-forge esmpy=7.1.0 xesmf cdsapi cfgrib xskillscore matplotlib


DirectoryNotACondaEnvironmentError: The target directory exists, but it is not a conda environment.
Use 'conda create' to convert the directory to a conda environment.
  target directory: /usr/local




In [4]:
!pip uninstall -y shapely;    # cartopy and shapely aren't friends (early 2020)
!pip install shapely --no-binary shapely;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.9/199.9 kB 3.2 MB/s eta 0:00:005.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
anceled
ERROR: Operation cancelled by user


In [5]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

## 2. Load the data
Let's start importing all the needed Python modules. 

In [6]:
import xarray as xr             # to open and manipulate data arrays 
import xskillscore as xs        # calculating skill scores
import xesmf as xe              # regridding
import numpy as np
      
import os
import os.path
from pathlib import Path

ModuleNotFoundError: No module named 'xskillscore'

In [ ]:
# Plot and mapping
import matplotlib.pyplot as plt
#import cartopy._crs as ccrs
#import cartopy
# Increase the size of the output figures
plt.rcParams["figure.figsize"] = (16,8)


In [ ]:
# To minimise the warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 2.1 Setting up the request for the CDS
If you don't have an account on the [Copernicus Data Store](https://cds.climate.copernicus.eu/#!/home) you need to [register and install the API](https://cds.climate.copernicus.eu/api-how-to)

**Important**: don't forget to insert your key in the following chunk of code

In [ ]:
!echo "url: https://cds.climate.copernicus.eu/api/v2" > $HOME/.cdsapirc
#!echo "key: {uid}:{api-key}" >> $HOME/.cdsapirc
!echo "key: 145:41b67f11-96a6-4f77-babd-4538caeab15c" >> $HOME/.cdsapirc

In [ ]:
import cdsapi

In [ ]:
# create a new client
c = cdsapi.Client()

### 2.2 Download climate data: seasonal forecasts
From the CDS you can download the [C3S Seasonal Forecasts](https://climate.copernicus.eu/seasonal-forecasts)

Let's define a list of the models and of the variables available

In [ ]:
MODELS = [
    {'centre': 'ecmwf', 'name': 'ECMWF', 'system': '5', 'start_year': 1993, 'end_year': 2016},
    {'centre': 'ukmo', 'name': 'UKMO', 'system': '14', 'start_year': 1993, 'end_year': 2016},
    {'centre': 'meteo_france', 'name': 'MF', 'system': '6', 'start_year': 1993, 'end_year': 2016},
    {'centre': 'dwd', 'name': 'DWD', 'system': '2', 'start_year': 1993, 'end_year': 2016},
    {'centre': 'cmcc', 'name': 'CMCC', 'system': '3', 'start_year': 1993, 'end_year': 2016}
]

VARS = [
    {'source': 'seasonal-monthly-single-levels', 'short_name': 'mslp', 'var': 'mean_sea_level_pressure', 'pressure_level': None, 'xarray_var':'msl'},
    {'source': 'seasonal-monthly-single-levels', 'short_name': 'u10', 'var': '10m_u_component_of_wind', 'pressure_level': None, 'xarray_var':'u10'},
    {'source': 'seasonal-monthly-single-levels', 'short_name': 'ws', 'var': '10m_wind_speed', 'pressure_level': None, 'xarray_var':'ws10'},
    {'source': 'seasonal-monthly-single-levels', 'short_name': 't2m', 'var': '2m_temperature', 'pressure_level': None, 'xarray_var':'t2m'},
    {'source': 'seasonal-monthly-single-levels', 'short_name': 'prec', 'var': 'total_precipitation', 'pressure_level': None, 'xarray_var':'tp'},
    {'source': 'seasonal-monthly-single-levels', 'short_name': 'ssr', 'var': 'surface_solar_radiation', 'pressure_level': None, 'xarray_var':'ssr'}
]

Now we select the starting month of the forecast, the model (selected from the `MODELS` dictionary) and the variable (from `VARS`)

In [ ]:
START_MONTH = 1
M  = MODELS[0]
V  = VARS[0]

Now we can prepare the dictionary containing the request for the CDS API. We will download the monthly seasonal forecast from the starting month for the next 4 months (from M+0 to M+3)

In [ ]:
request_dict = {
                'originating_centre': M['centre'],
                'variable':V['var'],
                'product_type':'monthly_mean',
                'year':[x for x in map(str, range(M['start_year'], M['end_year']))],
                'month': str(START_MONTH),
                'leadtime_month':[
                    '1', '2','3'
                ],
                'format':'grib',
                'system': M['system']
            }
print(request_dict)

The request is sent and the file downloaded if not already present. The download from the CDS **may take up to one hour**

In [ ]:
seasonal_target_filename = M['name']+'-'+V['short_name']+'-'+'S'+str(START_MONTH)+'-L1-3.grib'

if not Path(seasonal_target_filename).exists():
    print('Downloading ' + seasonal_target_filename)
    c.retrieve(
                V['source'],
                request_dict,
                seasonal_target_filename)
else:
    print('Using the file ' + seasonal_target_filename + ' already downloaded')

### 2.3 Download ERA5
Now we download the ERA5 data for the verification. 

In [ ]:
request_dict = {
                'variable':V['var'],
                'product_type':'monthly_averaged_reanalysis',
                'year':[x for x in map(str, range(M['start_year'], M['end_year']))],
                'month': [x for x in map(str, range(START_MONTH+1, START_MONTH+2+1))],
                'format':'grib',
                'time': '00:00'
            }
print(request_dict)

In [ ]:
era5_target_filename = 'era5-monthly-'+V['short_name']+'-'+str(M['start_year'])+'-'+str(M['end_year'])+'-'+'S'+str(START_MONTH)+'-L1-3.grib'
if not Path(era5_target_filename).exists():
    print('Downloading ' + era5_target_filename)
    c.retrieve(
                'reanalysis-era5-single-levels-monthly-means',
                request_dict,
                era5_target_filename)
else:
    print('Using the ERA5 file ' + era5_target_filename + ' already downloaded')

### 2.4 Regrid and compute
We regrid seasonal forecasts and reanalysis on the same grid. For the seasonal forecasts (`ftc`) we also compute the ensemble mean (`fct_final_ensmean`)

In [ ]:
# Defining the grid
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-89, 89, 1)),
                     'lon': (['lon'], np.arange(0, 360, 1))})

In [ ]:
fct = xr.open_dataset(seasonal_target_filename, engine='cfgrib').isel(step = slice(1,4)).mean(dim = 'step').rename({'latitude':'lat', 'longitude': 'lon'})

In [ ]:
  regridder = xe.Regridder(fct, ds_out, 'bilinear')

In [ ]:
fct_final = regridder(fct)

In [ ]:
fct_final_ensmean = fct_final.mean('number')

In [ ]:
fct_final_ensmean

In [ ]:
obs = xr.open_dataset(era5_target_filename, engine = 'cfgrib')

In [ ]:
obs_y = obs.groupby('time.year').mean('time').rename({'latitude':'lat', 'longitude': 'lon'})

In [ ]:
# REGRIDDING 1 DEGREE (OBS)
regridder = xe.Regridder(obs_y, ds_out, 'bilinear')

In [ ]:
obs_final = regridder(obs_y).rename({'year':'time'})

In [ ]:
obs_final['time'] = fct.time # this is needed because xskillscore expects 
print(obs_final)


## 3. Calculating metrics and skill scores
We compute in order:

1.    Pearson correlation between observations and ensemble mean
2.    Continuous Ranked Probability score (CRPS)
3.    Brier Skill Score for the upper tercile

All the skill scores are computed using `xskillscore` which uses `properscoring` for the probabilistic ones. 

In [ ]:
r = xs.pearson_r(obs_final, fct_final_ensmean, "time")

In [ ]:
#ax = plt.axes(projection=ccrs.Robinson())
ax = plt.axes()
#p = r[V['xarray_var']].plot(ax=ax, transform=ccrs.PlateCarree(), levels = [-1, 0, 0.3, 0.5, 0.6, 0.8, 0.9, 1], cmap = 'gnuplot')
p = r[V['xarray_var']].plot(ax=ax,  levels = [-1, 0, 0.3, 0.5, 0.6, 0.8, 0.9, 1], cmap = 'gnuplot')
#ax.set_global(); 
#ax.coastlines();

In [ ]:
crps_ensemble = xs.crps_ensemble(obs_final, fct_final.rename({'number':'member'}), dim ='time')

In [ ]:
#ax = plt.axes(projection=ccrs.Robinson())
ax = plt.axes()
#crps_ensemble[V['xarray_var']].mean('time').plot(ax=ax, transform=ccrs.PlateCarree())
crps_ensemble[V['xarray_var']].plot(ax=ax)
#ax.set_global(); ax.coastlines();

In [ ]:
brier_score = xs.brier_score(obs_final >  obs_final.quantile(2/3, dim = 'time'), (fct_final > fct_final.quantile(2/3, dim = 'time')).mean("number"), dim = 'time')


In [ ]:
baseline = xs.brier_score(obs_final >  obs_final.quantile(2/3, dim = 'time'), xr.DataArray(1/3), dim = 'time')

In [ ]:
#ax = plt.axes(projection=ccrs.Orthographic(20, 35))
#ax = plt.axes(projection=ccrs.Robinson())
ax = plt.axes()
#(1 - (brier_score/baseline)).mean(dim = 'time')[V['xarray_var']].plot(ax=ax, transform=ccrs.PlateCarree(), levels = [0, 0.01, 0.05, 0.1, 0.2, 0.3,  1], cmap = 'YlOrBr')
(1 - (brier_score/baseline))[V['xarray_var']].plot(ax=ax, levels = [0, 0.01, 0.05, 0.1, 0.2, 0.3,  1], cmap = 'YlOrBr')
#ax.set_global(); ax.coastlines();